In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import  mean_squared_error,r2_score

In [3]:
carsell=pd.read_csv("Car Sell Dataset.csv")

In [4]:
carsell.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140904 entries, 0 to 140903
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Brand          140904 non-null  object
 1   Model Name     140904 non-null  object
 2   Model Variant  140904 non-null  object
 3   Car Type       140904 non-null  object
 4   Transmission   140904 non-null  object
 5   Fuel Type      140904 non-null  object
 6   Year           140904 non-null  int64 
 7   Kilometers     140904 non-null  int64 
 8   Owner          140904 non-null  object
 9   State          140904 non-null  object
 10  Accidental     140904 non-null  object
 11  Price          140904 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 12.9+ MB


In [5]:
carsell.isnull().mean()*100


Brand            0.0
Model Name       0.0
Model Variant    0.0
Car Type         0.0
Transmission     0.0
Fuel Type        0.0
Year             0.0
Kilometers       0.0
Owner            0.0
State            0.0
Accidental       0.0
Price            0.0
dtype: float64

In [25]:
x=carsell.drop(columns=["Price"],axis=1)
y=carsell["Price"]

x_train_original,x_test_original,y_train_original,y_test_original=tt(x,y,test_size=0.2,random_state=0)

In [26]:
print("shape of x_train_original:", x_train_original.shape)
print("shape of x_test_original:", x_test_original.shape)
print("shape of y_train_original:", y_train_original.shape)
print("shape of y_test_original:", y_test_original.shape)

shape of x_train_original: (112723, 11)
shape of x_test_original: (28181, 11)
shape of y_train_original: (112723,)
shape of y_test_original: (28181,)


In [27]:
print(x_train_original)

                Brand Model Name Model Variant   Car Type Transmission  \
52445            Tata     Altroz            XM  Hatchback       Manual   
131547          Honda       City             S      Sedan       Manual   
98448        Mahindra     Bolero           AX3        SUV       Manual   
83867   Maruti Suzuki     Baleno          Base  Hatchback       Manual   
114017  Maruti Suzuki      Dzire             V      Sedan       Manual   
...               ...        ...           ...        ...          ...   
41993   Maruti Suzuki      Dzire             Z      Sedan       Manual   
97639            Ford   EcoSport      Ambiente        SUV       Manual   
95939          Toyota     Innova            ZX        MPV       Manual   
117952        Hyundai        i10            SX  Hatchback       Manual   
43567          Toyota     Glanza             V  Hatchback       Manual   

       Fuel Type  Year  Kilometers Owner           State Accidental  
52445     Diesel  2023       72535   1st 

In [28]:
obj= x_train_original.select_dtypes(include=['object']).columns.tolist()
num= x_train_original.select_dtypes(include=['int64']).columns.tolist()
print("object columns:", obj)
print("numerical columns:", num)

object columns: ['Brand', 'Model Name', 'Model Variant', 'Car Type', 'Transmission', 'Fuel Type', 'Owner', 'State', 'Accidental']
numerical columns: ['Year', 'Kilometers']


In [29]:
tra= ColumnTransformer(
    transformers=[
        ("Scaled",StandardScaler(),num),
        ("Encoded",OneHotEncoder(sparse_output=False,drop="first"),obj)
    ],remainder='passthrough'
)

In [30]:
X_train=tra.fit_transform(x_train_original)
X_test=tra.transform(x_test_original)
print("Final refined data of X_train:",X_train)
print("Final refined data of X_test:",X_test)

Final refined data of X_train: [[ 1.18064664 -0.4577259   0.         ...  0.          0.
   0.        ]
 [-2.34650333  1.16669799  0.         ...  0.          0.
   0.        ]
 [-0.9748339  -0.45544669  0.         ...  0.          0.
   0.        ]
 ...
 [-1.17078668  0.53617146  0.         ...  0.          0.
   0.        ]
 [-1.758645   -0.97219599  0.         ...  0.          0.
   0.        ]
 [ 0.59278831  0.99860642  0.         ...  0.          1.
   0.        ]]
Final refined data of X_test: [[ 0.39683553  1.41254717  0.         ...  0.          0.
   0.        ]
 [ 0.00492998  1.57756588  0.         ...  0.          0.
   0.        ]
 [-1.758645   -1.32252242  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.98469386  0.35198299  0.         ...  0.          0.
   0.        ]
 [-0.9748339   0.97426774  0.         ...  0.          0.
   0.        ]
 [ 1.18064664  0.55821058  0.         ...  0.          0.
   0.        ]]


In [31]:
lr=LinearRegression()
lr.fit(X_train,y_train_original)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [32]:
y_pred=lr.predict(X_test)
print(y_pred)

[684039.34606497 553345.37362907 677601.86446863 ... 896031.8631466
 369097.16591268 731591.58443063]


In [33]:
y_test_original

57964      404250
122941     627627
67378      632491
133070     178211
17137     1146011
           ...   
72188     1386000
9925       942210
60184      726000
44039      374343
116604     376200
Name: Price, Length: 28181, dtype: int64

In [34]:
print(r2_score(y_pred,y_test_original))
print(np.sqrt(mean_squared_error(y_pred,y_test_original)))

-1.5932322619430193
378170.42997363803


In [35]:
pipe=Pipeline(steps=[("Transformer",tra),("Model",lr)])

In [36]:
pipe.fit(x_train_original,y_train_original)


,steps,"[('Transformer', ...), ('Model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('Scaled', ...), ('Encoded', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [37]:
joblib.dump(pipe,"carsell_model.pkl")

['carsell_model.pkl']

In [38]:
carsell["Year"].max()

np.int64(2023)

In [39]:
carsell["Model Variant"]

0           AX5
1         Style
2             Z
3         Magna
4            W8
          ...  
140899      W11
140900      Era
140901        V
140902        E
140903     Base
Name: Model Variant, Length: 140904, dtype: object

In [40]:
carsell["Year"].min()

np.int64(2000)